<a href="https://colab.research.google.com/github/run-llama/llama_index/blob/main/docs/docs/examples/vector_stores/MongoDBAtlasVectorSearch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="在 Colab 中打开"/></a>


## MongoDB Atlas


如果您在Colab上打开这个笔记本，您可能需要安装LlamaIndex 🦙。


In [ ]:
%pip install llama-index-vector-stores-mongodb

In [ ]:
!pip install llama-index

In [ ]:
# 提供URI给构造函数，或使用环境变量
import pymongo
from llama_index.vector_stores.mongodb import MongoDBAtlasVectorSearch
from llama_index.core import VectorStoreIndex
from llama_index.core import StorageContext
from llama_index.core import SimpleDirectoryReader

下载数据


In [ ]:
!mkdir -p 'data/10k/'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10k/uber_2021.pdf' -O 'data/10k/uber_2021.pdf'

In [ ]:
# mongo_uri = os.environ["MONGO_URI"]
mongo_uri = (
    "mongodb+srv://<username>:<password>@<host>?retryWrites=true&w=majority"
)
mongodb_client = pymongo.MongoClient(mongo_uri)
store = MongoDBAtlasVectorSearch(mongodb_client)
storage_context = StorageContext.from_defaults(vector_store=store)
uber_docs = SimpleDirectoryReader(
    input_files=["./data/10k/uber_2021.pdf"]
).load_data()
index = VectorStoreIndex.from_documents(
    uber_docs, storage_context=storage_context
)

In [ ]:
response = index.as_query_engine().query("What was Uber's revenue?")
display(Markdown(f"<b>{response}</b>"))

<b>
Uber's revenue for 2021 was $17,455 million.</b>

In [ ]:
from llama_index.core import Response

# 初始大小
print(store._collection.count_documents({}))
# 获取ref_doc_id
typed_response = (
    response if isinstance(response, Response) else response.get_response()
)
ref_doc_id = typed_response.source_nodes[0].node.ref_doc_id
print(store._collection.count_documents({"metadata.ref_doc_id": ref_doc_id}))
# 测试存储删除
if ref_doc_id:
    store.delete(ref_doc_id)
    print(store._collection.count_documents({}))

4454
1
4453


注意：对于MongoDB Atlas，您还需要额外创建一个Atlas搜索索引。

[MongoDB文档 | 创建Atlas矢量搜索索引](https://www.mongodb.com/docs/atlas/atlas-vector-search/create-index/)
